In [1]:
const dotenv = require('dotenv')
const axios = require('axios')
const crypto = require('crypto')
const rateLimit = require('axios-rate-limit')
const http = rateLimit(axios.create(), { maxRPS: 1})

In [2]:
dotenv.config({verbose: false});

{
  parsed: {
    VUE_APP_BINANCE_API_KEY: '2PMRdBVcByWs8eKKZVF80VbwL8q6dojBt72SyzsGAOWJowqSIHYTGed7FfRIoMGK',
    VUE_APP_BINANCE_API_SECRET: 'iHqnsMpuxOfSCdEA9BKJvp63KDuvBkbKKe8XHHSGJaubLJelvkYfZK3egU0kQMsw',
    VUE_APP_GATEIO_API_KEY: '498d976daf238a8d5541ff475990033b',
    VUE_APP_GATEIO_API_SECRET: '68488b65de9c39cb4aa1895f49caa0899e3febeee61c4bbe7717213ddecd9b82',
    API_KEYS: '{ "gate":{"key": "498d976daf238a8d5541ff475990033b" ,"secret":"68488b65de9c39cb4aa1895f49caa0899e3febeee61c4bbe7717213ddecd9b82"}, "binance":{"key": "2PMRdBVcByWs8eKKZVF80VbwL8q6dojBt72SyzsGAOWJowqSIHYTGed7FfRIoMGK", "secret":"iHqnsMpuxOfSCdEA9BKJvp63KDuvBkbKKe8XHHSGJaubLJelvkYfZK3egU0kQMsw"}}'
  }
}

In [3]:
const keys = (JSON.parse(process.env.API_KEYS))

In [4]:
async function portfolioData() {
  await return new Promise((resolve, reject) => {
    const binanceAPI = "https://api.binance.com";
    const binanceEndpoint = "/sapi/v1/accountSnapshot";
    const gateAPI = "https://api.gateio.ws";
    const gateEndpoint = "/api/v4/spot/accounts";

    const proxyUrl = "https://thingproxy.freeboard.io/fetch/";

    let data = {
      gate: [],
      binance: [],
      combined: [],
    };

    let binanceRequestData = {};
    binanceRequestData.queryString = `type=SPOT&timestamp=${Date.now()}`;
    binanceRequestData.requestString = encodeURIComponent(
      `?${binanceRequestData.queryString}&signature=${binanceSignature(
        binanceRequestData.queryString
      )}`
    );

    const gateRequest = {
      method: "GET",
      url: `${proxyUrl}${gateAPI}${gateEndpoint}`,
      headers: gateSignature("GET", gateEndpoint, "", ""),
    };

    const binanceRequest = {
      method: "GET",
      url: `${proxyUrl}${binanceAPI}${binanceEndpoint}${binanceRequestData.requestString}`,
      headers: { "X-MBX-APIKEY": keys.binance.key },
    };

    axios
      .all([axios(gateRequest), axios(binanceRequest)])
      .then(
        axios.spread((val1, val2) => {
          val1.data.forEach((x) => {
            if(x.currency !== 'USDTEST')
              data.gate.push({
                symbol: x.currency,
                balance: x.available,
                exchange: 'gate'
              })
            })
            
          val2.data.snapshotVos
            .slice(-1)
            .pop()
            .data.balances.forEach((x) => {
              data.binance.push({
                symbol: x.asset,
                balance: x.free,
                exchange: 'binance'
              })
            });
          
          resolve({
              data: [...data.binance, ...data.gate]
          });
        })
      )
      .catch((err) => reject(err));

    function gateSignature(method, url, queryString, payloadString) {
      const hmac = crypto.createHmac("sha512", keys.gate.secret);
      const hash = crypto.createHash("sha512");

      const timestamp = Math.floor(Date.now() / 1000);

      const hashed = hash.update(payloadString || "").digest("hex");
      const sig = `${method}\n${url}\n${queryString}\n${hashed}\n${timestamp}`;
      const signed = hmac.update(sig).digest("hex");

      return { KEY: keys.gate.key, Timestamp: timestamp, SIGN: signed };
    }

    function binanceSignature(queryString) {
      return crypto
        .createHmac("sha256", keys.binance.secret)
        .update(queryString)
        .digest("hex");
    }
  });
}

In [5]:
async function transform(data) {
  const geckoAPI = "https://api.coingecko.com/api/v3";
  const geckoAll = "/coins/list?include_platform=false";
  const market   = "/coins/%s/market_chart/range?vs_currency=usd&from=%s&to=%s"
  const coin     = "/coins/%s?localization=false&tickers=false&market_data=false&community_data=false&developer_data=false&sparkline=false"
  const coins    = await http.get(`${geckoAPI}${geckoAll}`);

  // filter all coins from coingecko with portfolio data using symbol value
  // ary = portfolio [...symbols] array, ['ETH', 'BTC', 'ADA']

  let ary = Object.values(data.data).map((el, i, ar) => ar[i] = el.symbol.toUpperCase())
  let filtered = coins.data.filter((x) => {
    return ary.includes(x.symbol.toUpperCase()) || x.id == 'bit_financial' || x.id == 'bitcoin-free-cash'
  })

  // add coin id and name to portfolio data

  data.data.forEach((i, idx, ar) => {
    filtered.forEach(j => {
      if(i.symbol.toUpperCase() === j.symbol.toUpperCase()) {
        Object.assign(ar[idx], {id: j.id, name: j.name})
      }
    })
  })


  // filter out undefined data, only e.g. binance leverage tokens & fiat currencies
  // --> data iteration: get coin data
  // {imageurl: coingecko.imgurl, labels: [...labels], prices: [...prices]} etc etc
  // timespan change: adjust number in d2.

  data = data.data
  .filter((x) => {
    return x.id !== undefined
  })
  .map(async (x) => {
    let d1 = new Date()  
    let d2 = new Date().setDate(new Date().getDate() - 7)

    let marketQuery = util.format(market, x.id, d2 / 1000, d1 / 1000)
    let coinQuery   = util.format(coin,   x.id)

    let marketData = await http.get(geckoAPI + marketQuery)
    let coinData   = await http.get(geckoAPI + coinQuery)
    
    let returnData = {
      id:         x.id,
      symbol:     x.symbol,
      balance:    x.balance,
      imageUrl:   coinData.data.image.large.split("?")[0],
      marketData: marketData.data,
      exchange:   x.exchange
    }
    return returnData
  })

  return data
}

In [6]:
portfolioData().then((val) => {
  transform(val).then((val) => {
    Promise.all(val).then(val => {
      console.log(val)
    })
  })
})

[
  {
    id: 'cardano',
    symbol: 'ADA',
    balance: '90.7784',
    imageUrl: 'https://assets.coingecko.com/coins/images/975/large/cardano.png',
    marketData: { prices: [Array], market_caps: [Array], total_volumes: [Array] },
    exchange: 'binance'
  },
  {
    id: 'singularitynet',
    symbol: 'AGI',
    balance: '0.051',
    imageUrl: 'https://assets.coingecko.com/coins/images/2138/large/singularitynet.png',
    marketData: { prices: [Array], market_caps: [Array], total_volumes: [Array] },
    exchange: 'binance'
  },
  {
    id: 'aion',
    symbol: 'AION',
    balance: '0.008',
    imageUrl: 'https://assets.coingecko.com/coins/images/1023/large/Aion_Currency_Symbol_Transparent_PNG.png',
    marketData: { prices: [Array], market_caps: [Array], total_volumes: [Array] },
    exchange: 'binance'
  },
  {
    id: 'ankr',
    symbol: 'ANKR',
    balance: '981.018',
    imageUrl: 'https://assets.coingecko.com/coins/images/4324/large/U85xTl2.png',
    marketData: { prices: [Array], m